In [1]:
import matplotlib.pyplot as plt
def visMatr(matrix):
    plt.imshow(matrix, cmap='viridis', interpolation='nearest')
    plt.colorbar()
    plt.title('Matrix Visualization')
    plt.show()

In [2]:
import os
import torch
import torch.nn as nn
from model import Transformer, ModelParameters
from data.tokenizer import RegexTokenizer

models_folder = "models"
tokenizer_folder = "data"
model_name = "alilama_29M"

tok_path = os.path.join("data", "tokenizer.model")
try:
    tokenizer = RegexTokenizer()
    tokenizer.load(tok_path)
except:
    raise RuntimeError(f"tokenizer not found. Make sure {tok_path} exists")

device = "cpu"
token_count = len(tokenizer)
model_path = os.path.join(models_folder, model_name+".pth")

if os.path.exists(model_path):
    loaded_state = torch.load(model_path, map_location=device)
    inpt_params = loaded_state["params"]
    params = ModelParameters(**inpt_params)
    print("parameters:", inpt_params)
    params.max_seq_len = 10000 # increase as seen in ALiBi paper
    model = Transformer(token_count=token_count, device=device, params=params)
    model.load_state_dict(loaded_state['state_dict'])
    print("Model loaded successfully.")
else:
    print("Could not find model!")

def generateStream(size, inpt_tokens=None, temperature=1):
    for res in model.genLazy(size, inpt_tokens=inpt_tokens, temperature=temperature):
        l = [res[0][-1]]
        yield tokenizer.decode(l)

Could not find model!


In [4]:
# this is the story the model tries to continue:
start_story = "When i was a little kid, i always wanted to have some nice warm Gloves." # Don't use a trailing space!!!
# REASONING TEST (sometimes works). We expect something like "come down.":
# start_story = 'If i throw a ball into the air, it will eventually'
modelInput = " EOR " + start_story
temperature = 5 # how deterministic the model decides the next token (0 = random)
model.train()
as_tens = tokenizer.encode_ordinary(modelInput)
print(len(as_tens))
as_tens = torch.tensor(as_tens).unsqueeze(0).to(device)
for l in generateStream(50, as_tens, temperature=temperature):
    print(l, end="")

# show attention scores
"""att_scores = model.getAttScores()
first_block = att_scores[0]
for head in range(first_block.shape[1]):
    att = first_block[0,head,:,:].detach().numpy()
    visMatr(att)"""

print("")

20
 I was so excited to get these. They are very soft and warm, and the fur is very soft. I am very happy with them. EOR I love these! They are so soft and warm. I have a hard time finding boots that fit
